#  Comparison of Hg Lamp and Laser stability

In [2]:
import numpy as np
import pandas as pd
import skimage.io
import bokeh.io
import bokeh.plotting
import glob
import sys
sys.path.insert(0, '../../../')
import mwc.viz
colors = mwc.viz.set_plotting_style()
bokeh.io.output_notebook()

Loading BokehJS ...

## The problem 

I want to directly compare the stability in time and space for the Hg lamp as well as the mCherry laser. To do so, I took several data sets on 20180201 of a fluorescent slide being illuminated by the mCherry laser line. On 20180206, I took comparable measurements using an Hg lamp as an illumination source.  

## Loading the data

In [71]:
# Define the data directories
data_dir = '../../../data/images'
lamp_files = np.sort(glob.glob('{0}/20180206_hg*/*test_0*/*mCherry*.tif'.format(data_dir)))
laser_files = np.sort(glob.glob('{0}/20180207_laser*/*mcherry_single_pos_1/*Pos0/*.tif'.format(data_dir)))

# Load each image set
lamp_ims = skimage.io.ImageCollection(lamp_files)
laser_ims = skimage.io.ImageCollection(laser_files)

# Compute the means.
mean_lamp = np.array([np.mean(im.astype(np.uint16)) for im in lamp_ims])
mean_laser = np.array([np.mean(im.astype(np.uint16)) for im in laser_ims])
mean_lamp = mean_lamp / np.max(mean_lamp)
mean_laser = mean_laser / np.max(mean_laser)
# Normalize each to the mean. 
lamp_norm = mean_lamp - np.mean(mean_lamp)
laser_norm = mean_laser - np.mean(mean_laser)

In [72]:
# Plot each one. 
p = mwc.viz.boilerplate(x_axis_label='frame number',
                       y_axis_label='normalized intensity')
frames = np.arange(0, len(lamp_norm))
p.line(frames, lamp_norm, color='tomato', legend='Hg lamp')
p.line(frames, laser_norm, color='dodgerblue', legend='laser')
bokeh.io.show(p)

In [73]:
diff_laser = np.sqrt((np.diff(mean_laser) / mean_laser[:-1])**2)
diff_lamp = np.sqrt((np.diff(mean_lamp) / mean_lamp[:-1])**2)

In [74]:
p = mwc.viz.boilerplate(x_axis_label='time step', y_axis_label='percent difference')
p.line(frames[:-1], diff_laser * 100, color='dodgerblue', legend='laser' )
p.line(frames[:-1], diff_lamp * 100, color='tomato', legend='lamp' )
bokeh.io.show(p)

In [75]:
# Look at the distribution of differences
lamp_x, lamp_y = np.sort(diff_lamp * 100), np.arange(len(diff_lamp)) / len(diff_lamp)
laser_x, laser_y = np.sort(diff_laser * 100), np.arange(len(diff_laser)) / len(diff_laser)

p = mwc.viz.boilerplate(x_axis_label='percent difference', y_axis_label='cumulative distribution',
                       plot_width=800)
p.circle(lamp_x, lamp_y, color='tomato', legend='lamp')
p.circle(laser_x, laser_y, color='dodgerblue', legend='laser')
bokeh.io.show(p)

In [77]:
var_lamp = np.var(diff_lamp*100)
var_laser = np.var(diff_laser*100)
avg_lamp = np.mean(diff_lamp*100)
avg_laser = np.mean(diff_laser*100)
fano_lamp = var_lamp / avg_lamp
fano_laser = var_laser / var_lamp
print(fano_lamp, fano_laser)

0.0821651783437 16.7590187156


In [78]:
avg_lamp

0.11623255109093981

In [79]:
avg_laser

0.44611136628861842